In [1]:
library(tidyverse)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


In [2]:
options(repr.matrix.max.rows=10)

### Load Data

In [3]:
results = read_csv("build/eval-results.csv")
user.results = read_csv("build/eval-users-results.csv")
results
user.results

Parsed with column specification:
cols(
  DataSet = col_character(),
  Partition = col_integer(),
  Algorithm = col_character(),
  Succeeded = col_character(),
  BuildTime = col_double(),
  TestTime = col_double(),
  RMSE.ByUser = col_double(),
  RMSE.ByRating = col_double(),
  MRR = col_double(),
  Precision = col_double(),
  Recall = col_double(),
  F1 = col_double()
)
Parsed with column specification:
cols(
  DataSet = col_character(),
  Partition = col_integer(),
  Algorithm = col_character(),
  User = col_integer(),
  TestTime = col_double(),
  TestItems = col_integer(),
  RMSE = col_double(),
  Rank = col_integer(),
  RecipRank = col_double(),
  Precision = col_double(),
  Recall = col_double(),
  F1 = col_double()
)


DataSet,Partition,Algorithm,Succeeded,BuildTime,TestTime,RMSE.ByUser,RMSE.ByRating,MRR,Precision,Recall,F1
crossfold,1,Popular,Y,23.492,1569.459,3.347262,7.361003,0.05482688,0.01337847,0.03905692,0.01993012
crossfold,1,PF-Implicit,Y,114259.284,32815.254,4.073284,7.650870,0.11474263,0.02767972,0.07781151,0.04083374
crossfold,3,Popular,Y,19.363,1431.378,3.350605,6.509694,0.05484751,0.01341631,0.03905462,0.01997178
crossfold,3,PF-Implicit,Y,134254.735,33527.639,4.076266,6.835506,0.11557490,0.02795062,0.07642106,0.04093095
crossfold,4,Popular,Y,18.496,1582.754,3.356726,6.596970,0.05472843,0.01335821,0.03895122,0.01989387


DataSet,Partition,Algorithm,User,TestTime,TestItems,RMSE,Rank,RecipRank,Precision,Recall,F1
crossfold,1,Popular,844099,2.477,3,18.47807181,NA,0,0,0,0
crossfold,1,Popular,923802,2.475,1,7.00008633,NA,0,0,0,0
crossfold,1,Popular,79532,2.481,3,0.02633701,NA,0,0,0,0
crossfold,1,Popular,589189,2.489,3,0.15222460,NA,0,0,0,0
crossfold,1,Popular,286515,2.475,4,0.70742766,NA,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
crossfold,4,Popular,207084,0.019,5,2.84743673,NA,0,0,0,0
crossfold,4,Popular,207085,0.015,2,0.01083446,NA,0,0,0,0
crossfold,4,Popular,207086,0.013,4,2.13278396,NA,0,0,0,0
crossfold,4,Popular,207087,0.016,4,1.25656664,NA,0,0,0,0


In [4]:
test.ratings = read_csv("build/crossfold.out/part03.test.csv",
                col_names=c("user", "item", "rating", "timestamp"))
test.ratings

Parsed with column specification:
cols(
  user = col_integer(),
  item = col_integer(),
  rating = col_double(),
  timestamp = col_integer()
)


user,item,rating,timestamp
73250,3459,2,-1
629014,33717,2,-1
544771,7093,1,-1
215215,5901,1,-1
192687,6357,4,-1
⋮,⋮,⋮,⋮
66179,147948,1,-1
329331,106737,4,-1
763431,144466,1,-1
567968,37166,1,-1


In [5]:
user.test.item = test.ratings %>%
    group_by(user) %>%
    summarize(itemCount=n()) %>%
    ungroup %>%
    mutate(total=sum(itemCount))

In [6]:
user.test.item = user.test.item %>%
    rename(User=user)
user.test.item

User,itemCount,total
1,19,9159020
2,4,9159020
3,1,9159020
4,2,9159020
5,7,9159020
⋮,⋮,⋮
1019314,2,9159020
1019315,54,9159020
1019316,5,9159020
1019317,12,9159020


In [12]:
user.results.stats = user.results %>%
    filter(Algorithm=="PF-Implicit" & Partition==3) %>%
    inner_join(user.test.item)
user.results.stats

Joining, by = "User"


DataSet,Partition,Algorithm,User,TestTime,TestItems,RMSE,Rank,RecipRank,Precision,Recall,F1,itemCount,total
crossfold,3,PF-Implicit,366292,0.417,25,2.808400,NA,0.0000000,0.00,0.00,0.00000000,25,9159020
crossfold,3,PF-Implicit,398147,0.424,4,3.704680,3,0.3333333,0.05,0.25,0.08333333,4,9159020
crossfold,3,PF-Implicit,159178,0.446,5,2.827107,1,1.0000000,0.10,0.40,0.16000000,5,9159020
crossfold,3,PF-Implicit,79604,0.428,10,4.217140,NA,0.0000000,0.00,0.00,0.00000000,10,9159020
crossfold,3,PF-Implicit,589329,0.428,4,2.645585,NA,0.0000000,0.00,0.00,0.00000000,4,9159020
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
crossfold,3,PF-Implicit,796315,0.423,5,7.8849990,NA,0,0,0,0,5,9159020
crossfold,3,PF-Implicit,796316,0.425,2,7.5134683,NA,0,0,0,0,2,9159020
crossfold,3,PF-Implicit,796317,0.423,2,0.9984006,NA,0,0,0,0,2,9159020
crossfold,3,PF-Implicit,796318,0.428,5,1.6705680,NA,0,0,0,0,5,9159020


In [13]:
user.results.normed = user.results.stats %>%
    filter(TestItems < 20) %>%
    mutate(nPrecision = Precision * 20 / TestItems)
user.results.normed

DataSet,Partition,Algorithm,User,TestTime,TestItems,RMSE,Rank,RecipRank,Precision,Recall,F1,itemCount,total,nPrecision
crossfold,3,PF-Implicit,398147,0.424,4,3.704680,3,0.3333333,0.05,0.25,0.08333333,4,9159020,0.25
crossfold,3,PF-Implicit,159178,0.446,5,2.827107,1,1.0000000,0.10,0.40,0.16000000,5,9159020,0.40
crossfold,3,PF-Implicit,79604,0.428,10,4.217140,NA,0.0000000,0.00,0.00,0.00000000,10,9159020,0.00
crossfold,3,PF-Implicit,589329,0.428,4,2.645585,NA,0.0000000,0.00,0.00,0.00000000,4,9159020,0.00
crossfold,3,PF-Implicit,493765,0.407,3,3.107671,NA,0.0000000,0.00,0.00,0.00000000,3,9159020,0.00
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
crossfold,3,PF-Implicit,796315,0.423,5,7.8849990,NA,0,0,0,0,5,9159020,0
crossfold,3,PF-Implicit,796316,0.425,2,7.5134683,NA,0,0,0,0,2,9159020,0
crossfold,3,PF-Implicit,796317,0.423,2,0.9984006,NA,0,0,0,0,2,9159020,0
crossfold,3,PF-Implicit,796318,0.428,5,1.6705680,NA,0,0,0,0,5,9159020,0


In [14]:
user.results.normed2 = user.results.stats %>%
    filter(TestItems >= 20) %>%
    mutate(nPrecision = Precision)
user.results.normed2

DataSet,Partition,Algorithm,User,TestTime,TestItems,RMSE,Rank,RecipRank,Precision,Recall,F1,itemCount,total,nPrecision
crossfold,3,PF-Implicit,366292,0.417,25,2.808400,NA,0,0.0,0.0000000,0.0000000,25,9159020,0.0
crossfold,3,PF-Implicit,222882,0.420,38,4.713369,1,1,0.5,0.2631579,0.3448276,38,9159020,0.5
crossfold,3,PF-Implicit,923735,0.417,32,3.332727,NA,0,0.0,0.0000000,0.0000000,32,9159020,0.0
crossfold,3,PF-Implicit,159179,0.433,22,13.399468,NA,0,0.0,0.0000000,0.0000000,22,9159020,0.0
crossfold,3,PF-Implicit,334405,0.417,23,2.356757,NA,0,0.0,0.0000000,0.0000000,23,9159020,0.0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
crossfold,3,PF-Implicit,796253,0.429,36,2.146912,NA,0.0000000,0.00,0.00000000,0.00000000,36,9159020,0.00
crossfold,3,PF-Implicit,796272,0.428,27,3.661860,7,0.1428571,0.05,0.03703704,0.04255319,27,9159020,0.05
crossfold,3,PF-Implicit,796292,0.429,20,2.628067,1,1.0000000,0.05,0.05000000,0.05000000,20,9159020,0.05
crossfold,3,PF-Implicit,796301,0.432,35,2.647278,5,0.2000000,0.15,0.08571429,0.10909091,35,9159020,0.15


In [15]:
user.results.precision = bind_rows(user.results.normed, user.results.normed2) %>%
    group_by("Partition", "Algorithm") %>%
    summarize(nPrecision = mean(nPrecision)) %>%
    ungroup
user.results.precision

"""Partition""","""Algorithm""",nPrecision
Partition,Algorithm,0.07962644
